<center>
<img src="http://corpuslg.org/lael_english/wp-content/uploads/2020/04/lael_50_years_narrow_white.png.400px_300dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Images function set

## Importing the required libraries

In [1]:
import subprocess
from IPython.display import clear_output
import os
from pathlib import Path
import pandas as pd
from google.cloud import storage
from google.cloud import vision

## Preamble (Do not execute)

- Group number definition
- Selects the syntax of the the commands 'shuf' and 'sed' for Mac or Linux platforms
- The variables 'myshuf' and 'mysed' are not used in the programme
- Decision: comment it out

In [ ]:
#enter your group number
#group=4

#enter the label for your system, mac or linux
#mysystem=mac
#mysystem=linux

#  if [ "$mysystem" == mac ]
#    then
#    myshuf=gshuf
#    mygsed=gsed
#  else
#    myshuf=shuf
#    mygsed=sed
#  fi


## 'presample' function

- Processes 'tweets/scraped.txt' into 'images/presample.txt'
- Correction to create the folder 'images' for the first tine

### Original function (Do not execute)

In [ ]:
presample () {

mkdir -p images

grep fullUrl tweets/scraped.txt | cut -f2- | nl | sed 's/^[ ]*//' > images/presample.txt

}


### Executable from this notebook

In [2]:
subprocess.run(['wsl', './presample.sh'], capture_output = True)

CompletedProcess(args=['wsl', './presample.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J', stderr=b'')

## 'collecturls' function

- Processes 'images/presample.txt' into 'images/images_index.txt'
- Defines 13 image batches with 1000 images in each batch
- Creates one folder for each batch in 'images/images'
- The commands 'rg' and 'jq' are required
- Correction to resolve folder naming conflict with 'grabimages' function
    - from: 'sort z | uniq > folders'
    - to: 'sort z | uniq | sed 's/fl://' > folders'

### Original function (Do not execute)

In [ ]:
getimagesurls () {

last=$( cat images/presample.txt | wc -l | tr -dc '[0-9]' )

#mkdir -p images

rm -f images/urls.txt  ### WATCH THIS!

for i in $(eval echo {1..$last});
do
        rg -m1 "^"$i"	" images/presample.txt | jq '.' > z
        file=$( grep -m1 'fullUrl' z | cut -d'"' -f4  )
        format=$( echo $file | tr '?&=' '|' | cut -d'|' -f3 )
        id=$( grep -m1 'id"' z | tr '~' ' ' | sed -e 's/^[ ]*//' -e 's/:/~/' | cut -d'~' -f2 | tr -dc '[0-9]' )
        username=$( grep -m1 'username"' z | tr '~' ' ' | sed -e 's/^[ ]*//' -e 's/:/~/' | cut -d'~' -f2 | sed -e 's/"//' -e 's/^[ ]*//' -e 's/",$//' | tr '[:upper:]' '[:lower:]' )
        date=$( grep -m1 'date"' z | tr '~' ' ' | sed -e 's/^[ ]*//' | cut -d'~' -f2 | cut -d'T' -f1 | tr -dc '[0-9-]' )

        echo "---- collecturls $i / $last ----"

        echo "id:"$id"|d:$date|u:"$username"|i:$file|f:$format" >> images/urls.txt
done 

grep 'id:...................|' images/urls.txt | nl -nrz | sed 's/^/t:/' | tr '\t' '|' > w

for n in `seq -w 1 13`  # how many batches
do
  printf "$n\n%.0s" {1..1000}  # how many lines in each batch
done | sed 's/^/fl:/' > z

paste z w | tr '\t' '|' | grep 'id:' > images/images_index.txt

#sort z | uniq > folders
sort z | uniq | sed 's/fl://' > folders
while read folder
do
    mkdir -p images/images/$folder
done < folders

}


### Executable from this notebook

In [3]:
subprocess.run(['wsl', './getimagesurls.sh'], capture_output = True)

CompletedProcess(args=['wsl', './getimagesurls.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J---- collecturls 1 / 20 ----\n---- collecturls 2 / 20 ----\n---- collecturls 3 / 20 ----\n---- collecturls 4 / 20 ----\n---- collecturls 5 / 20 ----\n---- collecturls 6 / 20 ----\n---- collecturls 7 / 20 ----\n---- collecturls 8 / 20 ----\n---- collecturls 9 / 20 ----\n---- collecturls 10 / 20 ----\n---- collecturls 11 / 20 ----\n---- collecturls 12 / 20 ----\n---- collecturls 13 / 20 ----\n---- collecturls 14 / 20 ----\n---- collecturls 15 / 20 ----\n---- collecturls 16 / 20 ----\n---- collecturls 17 / 20 ----\n---- collecturls 18 / 20 ----\n---- collecturls 19 / 20 ----\n---- collecturls 20 / 20 ----\n', stderr=b'')

## 'grabimages' function

- Processes 'images/images_index.txt' into ''
- The command 'curl' is required

### Original function (Do not execute)

In [ ]:
grabimages () {

last=$( cat images/images_index.txt | wc -l | tr -cd '[0-9]' )

for i in $(eval echo {1..$last});
do
    sed -n "$i"p images/images_index.txt > z
    folder=$( cut -d'|' -f1 z | sed 's/fl://' )
    n=$( cut -d'|' -f2 z | sed 's/t://' )
    id=$( cut -d'|' -f3 z | sed 's/id://' )
    file=$( cut -d'|' -f6 z | sed 's/i://' )
    ext=$( cut -d'|' -f7 z | sed 's/f://' )
    
    echo "---- fetching image $n / $last ----"
    
    curl -k "$file" > images/images/"$folder"/"$n"."$ext"
done 

}


### Executable from this notebook

In [4]:
subprocess.run(['wsl', './grabimages.sh'], capture_output = True)

CompletedProcess(args=['wsl', './grabimages.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J---- fetching image 000001 / 20 ----\n---- fetching image 000002 / 20 ----\n---- fetching image 000003 / 20 ----\n---- fetching image 000004 / 20 ----\n---- fetching image 000005 / 20 ----\n---- fetching image 000006 / 20 ----\n---- fetching image 000007 / 20 ----\n---- fetching image 000008 / 20 ----\n---- fetching image 000009 / 20 ----\n---- fetching image 000010 / 20 ----\n---- fetching image 000011 / 20 ----\n---- fetching image 000012 / 20 ----\n---- fetching image 000013 / 20 ----\n---- fetching image 000014 / 20 ----\n---- fetching image 000015 / 20 ----\n---- fetching image 000016 / 20 ----\n---- fetching image 000017 / 20 ----\n---- fetching image 000018 / 20 ----\n---- fetching image 000019 / 20 ----\n---- fetching image 000020 / 20 ----\n', stderr=b'  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current\n                                 Dload  Upload   Total

## 'removedupes' function

- Removes duplicate images posted in the same message or in repeated messages by the same user (same message id)
- The function was empirically tested by manually introducing a duplicate image-containing message in 'tweets/scraped.txt' and it works as expected. It returns the following error message that apparently does not harm its functionality
    - './removedupes.sh: line 27: remove: No such file or directory\ngrep: remove: No such file or directory\nfind: paths must precede expression: `empty'\n'
- When there are no duplicates, the function ends with the following error message. This result is not harmful because it proves that there are no duplicates
    - './removedupes.sh: line 27: remove: No such file or directory\ngrep: remove: No such file or directory\nfind: paths must precede expression: `empty'\n'
- Correction to avoid 'images/images_index.txt' to have all lines excluded in case thare are no duplicates and, thus, the 'remove' file is empty
- Correction to avoid an error condition in case there are no duplicates and the file 'remove' is not generated
- Correction from 'find images/images -type f empty -exec rm {} +' to 'find images/images -type f -empty -exec rm {} +'
- Correction from 'cat i i f | sort | uniq -c | grep ' 2 ' | cut -c6- | grep -v '^$' | sed 's/^/t:/' > ionly' to 'cat i i f | sort | uniq -c | grep ' 2 ' | cut -c9- | grep -v '^$' | sed 's/^/t:/' > ionly'

### Original function (Do not execute)

In [ ]:
removedupes () {

# remove duplicate images posted in the same message or in repeated messages by the same user (same message id)
    
cut -d'|' -f3 images/images_index.txt | sed 's/id://' | sort | uniq -c | grep -v ' 1 ' | nl | sed 's/^[ ]*//' | tr '\t' ' ' | tr -s ' ' > d

rm -f remove

while read n maxhits id
do
    echo "--- listing $n ---"
    grep -m$maxhits $id images/images_index.txt | cut -d'|' -f6 | sort | uniq -d | sed 's/i://' > dupes
    
    while read dupe
    do
        grep $dupe images/images_index.txt | tail +2 | cut -d'|' -f2
    done < dupes >> remove
done < d 

# remove duplicate image files
#while read dupe
#do
#    pretty=$( echo $dupe | sed 's/t://' )
#    folder=$( grep $dupe images/images_index.txt | cut -d'|' -f1 | sed 's/fl://' )
#    ext=$( grep $dupe images/images_index.txt | cut -d'|' -f7 | sed 's/f://' )
#    rm -f images/images/"$folder"/"$pretty"."$ext"
#    echo "--- removing images/images/"$folder"/"$pretty"."$ext" ---"
#done < remove
if [[ -s remove ]]; then
    while read dupe
    do
        pretty=$( echo $dupe | sed 's/t://' )
        folder=$( grep $dupe images/images_index.txt | cut -d'|' -f1 | sed 's/fl://' )
        ext=$( grep $dupe images/images_index.txt | cut -d'|' -f7 | sed 's/f://' )
        rm -f images/images/"$folder"/"$pretty"."$ext"
        echo "--- removing images/images/"$folder"/"$pretty"."$ext" ---"
    done < remove
fi

# remove dupes from index
#grep -vf remove images/images_index.txt > z ; mv z images/images_index.txt
if [[ -s remove ]]; then
    grep -vf remove images/images_index.txt > z ; mv z images/images_index.txt
fi

# remove dupes from index again, for some reason some still remain
cut -d'|' -f2  images/images_index.txt | cut -d':' -f2 > i
find images/images -type f | cut -d'/' -f4 | cut -d'.' -f1 > f
cat i i f | sort | uniq -c | grep ' 2 ' | cut -c9- | grep -v '^$' | sed 's/^/t:/' > ionly
grep -vf ionly images/images_index.txt > b ; mv b images/images_index.txt

# remove empty image files
find images/images -type f -empty -exec rm {} +

# remove empty image files from index
cut -d'|' -f2  images/images_index.txt | cut -d':' -f2 > i
find images/images -type f | cut -d'/' -f4 | cut -d'.' -f1 > f
cat i i f | sort | uniq -c | grep ' 2 ' | cut -c9- | grep -v '^$' | sed 's/^/t:/' > ionly
grep -vf ionly images/images_index.txt > b ; mv b images/images_index.txt

}


### Executable from this notebook

In [5]:
subprocess.run(['wsl', './removedupes.sh'], capture_output = True)

CompletedProcess(args=['wsl', './removedupes.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J', stderr=b'')

## 'uploadtobucket' function

- Copies the 'images/images' folder to the user's Google Cloud Storage bucket

### Original function (Do not execute)

In [ ]:
uploadtobucket () {

# google cloud gsutil account = tonyberber@gmail.com

gcloud alpha storage cp -R images/images gs://socialmediaclassimages/group

}


### Executable from this notebook

#### Using Google Cloud SDK Shell (Do not execute)

- On your Desktop, click on 'Google Cloud SDK Shell'
- On the terminal window execute the commands as follows:

In [ ]:
# Go to the folder where the 'images' folder is located
C:\Users\eyamr\AppData\Local\Google\Cloud SDK>cd C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground
C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground>dir
 Volume in drive C is OS
 Volume Serial Number is FA6F-4E60

 Directory of C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground

20/09/2023  17:43    <DIR>          .
19/09/2023  15:29    <DIR>          ..
<omitted>
20/09/2023  17:37    <DIR>          images
<omitted>
              24 File(s)         51.289 bytes
               5 Dir(s)  134.018.973.696 bytes free

# List the buckets
C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground>gcloud storage ls
gs://laelimages/

# Run the following commad
C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground>gcloud alpha storage cp -R images/images gs://laelimages
Copying file://images\images\01\000001.jpg to gs://laelimages/images/01/000001.jpg
Copying file://images\images\01\000002.jpg to gs://laelimages/images/01/000002.jpg
  Completed files 2/2 | 159.3kiB/159.3kiB

Average throughput: 6.0MiB/s

C:\Users\eyamr\OneDrive\Documentos\0-Technology\Anaconda\playground>

#### Using the 'google-cloud-storage' Python library
Alternative 1: Completely equivalent to the Google Cloud SDK approach

In [6]:
def upload_directory_to_bucket(bucket_name, source_directory, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    for root, dirs, files in os.walk(source_directory):
        for file in files:
            local_path = os.path.join(root, file)
            blob_path = os.path.join(destination_blob_name, os.path.relpath(local_path, source_directory)).replace('\\', '/')
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_path)

    print(f'Directory {source_directory} uploaded to {bucket_name}/{destination_blob_name} successfully!')

end = False
while end == False:
    my_bucket = str(input('Enter your bucket name: '))
    if my_bucket != '':
        bucket_name = my_bucket
        end = True
        clear_output()

source_directory = r'.\images\images'
destination_blob_name = 'images'
upload_directory_to_bucket(bucket_name, source_directory, destination_blob_name)

Directory .\images\images uploaded to laelimages/images successfully!


#### Using the 'google-cloud-storage' Python library (Do not execute)
Alternative 2: Copying even empty subdirectories

In [ ]:
def upload_directory_to_bucket(bucket_name, source_directory, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    for root, dirs, files in os.walk(source_directory):
        for file in files:
            local_path = os.path.join(root, file)
            blob_path = os.path.join(destination_blob_name, os.path.relpath(local_path, source_directory)).replace('\\', '/')
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_path)

        for dir in dirs:
            dir_path = os.path.join(destination_blob_name, os.path.relpath(os.path.join(root, dir), source_directory)).replace('\\', '/')
            blob = bucket.blob(dir_path + '/')
            blob.upload_from_string('')

    print(f'Directory {source_directory} uploaded to {bucket_name}/{destination_blob_name} successfully!')

end = False
while end == False:
    my_bucket = str(input('Enter your bucket name: '))
    if my_bucket != '':
        bucket_name = my_bucket
        end = True
        clear_output()

source_directory = r'.\images\images'
destination_blob_name = 'images'
upload_directory_to_bucket(bucket_name, source_directory, destination_blob_name)

## 'googlelabels' function

- Detects the labels of the images with Google Vision

### Original function (Do not execute)

In [ ]:
googlelabels () {

# speed: 1 file per second

while read folder
do
  mkdir -p images/google_cloud/labels/$folder
  ### rm -f images/google_cloud/labels/$folder/*
done < folders

last=$( tail -1 images/images_index.txt | cut -d'|' -f2 | sed 's/t://'  )

for i in $(eval echo {1..$last});
do
    sed -n "$i"p images/images_index.txt > z
    folder=$( cut -d'|' -f1 z | sed 's/fl://' )
    n=$( cut -d'|' -f2 z | sed 's/t://' )
    id=$( cut -d'|' -f3 z | sed 's/id://' )
    file=$( cut -d'|' -f6 z | sed 's/i://' )
    ext=$( cut -d'|' -f7 z | sed 's/f://' )
    
    echo "---- detect-labels $i / $last ----"
    
    gcloud ml vision detect-labels --max-results=150  gs://socialmediaclassimages/group/"$folder"/"$n"."$ext" > images/google_cloud/labels/"$folder"/"$n".txt
    
done 

}


### Executable from this notebook

#### Using the 'google-cloud-vision' Python library
It is a Python code that is functionally equivalent to the original shell script (so it is a valid artefact) but it generates an output file format that may be incompatible with the subsequent shell scripts in the pipeline

In [8]:
def detect_labels(image_uri):
    """Detects labels in the image URL using the Google Cloud Vision API."""
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = image_uri
    response = client.label_detection(image = image, max_results = 150)
    if response.error.message:
        raise Exception(f'Error: {response.error.message}')
    return response.label_annotations

with open('folders', 'r') as f:
    for folder in f:
        folder = folder.strip()
        os.makedirs(f'images/google_cloud/labels/{folder}', exist_ok = True)
        # os.system(f'rm -f images/google_cloud/labels/{folder}/*')

last = subprocess.run(['wsl', 'tail', '-1', 'images/images_index.txt'], capture_output = True, text = True).stdout.strip().split('|')[1][2:]
last = int(last)

for i in range(1, last + 1):
    try:
        with open('images/images_index.txt', 'r') as f:
            line = next(line for j, line in enumerate(f, start = 1) if j == i)
            folder = line.split('|')[0][3:]
            n = line.split('|')[1][2:]
            id = line.split('|')[2][3:]
            file = line.split('|')[5][2:]
            ext = line.split('|')[6][2:5]

        print(f"---- detect-labels {i} / {last} ----")

        image_uri = f'gs://{bucket_name}/images/{folder}/{n}.{ext}'
        labels = detect_labels(image_uri)
        with open(f'images/google_cloud/labels/{folder}/{n}.txt', 'w') as f:
            for label in labels:
                f.write('description: ' + f'{label.description}\n')
                f.write('mid: ' + f'{label.mid}\n')
                f.write('score: ' + f'{label.score}\n')
                f.write('topicality: ' + f'{label.topicality}\n\n')
    except StopIteration:
        print('The iteration was stopped because there were empty files that have been removed.')

---- detect-labels 1 / 20 ----
---- detect-labels 2 / 20 ----
---- detect-labels 3 / 20 ----
---- detect-labels 4 / 20 ----
---- detect-labels 5 / 20 ----
---- detect-labels 6 / 20 ----
---- detect-labels 7 / 20 ----
---- detect-labels 8 / 20 ----
---- detect-labels 9 / 20 ----
---- detect-labels 10 / 20 ----
---- detect-labels 11 / 20 ----
---- detect-labels 12 / 20 ----
---- detect-labels 13 / 20 ----
---- detect-labels 14 / 20 ----
---- detect-labels 15 / 20 ----
---- detect-labels 16 / 20 ----
---- detect-labels 17 / 20 ----
---- detect-labels 18 / 20 ----
---- detect-labels 19 / 20 ----
The iteration was stopped because there were empty files that have been removed.


## 'labeltypes' function

- Groups the descriptions of the labels of each image into a line of the file 'labels.txt' per image
- Correction: include '| tr -d '\r' |' in the pipeline to remove the carriage return character. This character causes errors in the format of the file 'labels.txt'

### Original function (Do not execute)

In [ ]:
labeltypes () {

rm -f images/labels.txt

last=$( cat images/images_index.txt | wc -l )

for i in $(eval echo {1..$last});
do    
    sed -n "${i}p" images/images_index.txt > z
    folder=$(cut -d'|' -f1 z | sed 's/fl://')
    n=$(cut -d'|' -f2 z | sed 's/t://')
    id=$(cut -d'|' -f3 z | sed 's/id://')
    date=$(cut -d'|' -f4 z | sed 's/d://')
    username=$(cut -d'|' -f5 z | sed 's/u://')

    echo "---- labeltypes $i / $last ---"

# Google Cloud Vision Labels in the format obtained from the original 'googlelabels' function (JSON format)
#    grep '"description":' images/google_cloud/labels/"$folder"/"$n".txt | cut -d':' -f2 | tr -d '"' | sed -e 's/^[ ]*//' | tr '\n' ' ' | tr -d '\r' | sed 's/, $//' | tr ' ' '_' | sed 's/,_/,/g' | tr -d "'" | tr '[A-Z]' '[a-z]' | sed "s/^/fl:$folder|t:$n|id:$id|d:$date|u:$username|l:/" >> images/labels.txt

# Google Cloud Vision Labels in the format obtained from the Python version of the 'googlelabels' function
    grep 'description:' images/google_cloud/labels/"$folder"/"$n".txt | cut -d':' -f2 | tr -d '"' | sed -e 's/^[ ]*//' | tr '\n' ', ' | tr -d '\r' | sed 's/,$//' | tr ' ' '_' | sed 's/,_/,/g' | tr -d "'" | tr '[A-Z]' '[a-z]' | sed "s/^/fl:$folder|t:$n|id:$id|d:$date|u:$username|l:/" >> images/labels.txt
    echo >> images/labels.txt
done

}


### Executable from this notebook

In [9]:
subprocess.run(['wsl', './labeltypes.sh'], capture_output = True)

CompletedProcess(args=['wsl', './labeltypes.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J---- labeltypes 1 / 19 ---\n---- labeltypes 2 / 19 ---\n---- labeltypes 3 / 19 ---\n---- labeltypes 4 / 19 ---\n---- labeltypes 5 / 19 ---\n---- labeltypes 6 / 19 ---\n---- labeltypes 7 / 19 ---\n---- labeltypes 8 / 19 ---\n---- labeltypes 9 / 19 ---\n---- labeltypes 10 / 19 ---\n---- labeltypes 11 / 19 ---\n---- labeltypes 12 / 19 ---\n---- labeltypes 13 / 19 ---\n---- labeltypes 14 / 19 ---\n---- labeltypes 15 / 19 ---\n---- labeltypes 16 / 19 ---\n---- labeltypes 17 / 19 ---\n---- labeltypes 18 / 19 ---\n---- labeltypes 19 / 19 ---\n', stderr=b'')

## 'toplabels' function

- Creates a list of the most frequent labels

### Original function (Do not execute)

In [ ]:
toplabels () {

cut -d'|' -f6 images/labels.txt | sed 's/l://' | tr ',' '\n' | sort | uniq -c | sort -nr | sed 's/^[ ]*//' | grep '[a-z]' | head -1000 | cut -d' ' -f2- | nl -nrz | sed 's/^/v/' | tr '\t' ' ' > images/selectedwords

cp images/selectedwords images/var_index.txt

}


### Executable from this notebook

In [10]:
subprocess.run(['wsl', './toplabels.sh'], capture_output = True)

CompletedProcess(args=['wsl', './toplabels.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J', stderr=b'')

## 'sas' function

- Creates data to be processed in SAS

### Original function (Do not execute)

In [ ]:
sas () {

mkdir -p images/sas

rm -f images/columns

cut -d'|' -f3,6 images/labels.txt > a

while read n word 
do
  echo "--- $n ---"
  rg -w $word a | cut -d'|' -f1 | sed -e 's/id://' -e "s/$/ "$n" 1/" >> images/columns 
done < images/selectedwords

sort images/columns | uniq > a ; mv a images/columns  # to avoid words whose accents were stripped to be duplicated in the same text ; SAS can't handle that

#cut -d' ' -f2 tweets/selectedwords | gwc -L 
#head -1 columns | cut -d' ' -f1 | gwc -L

cp images/columns images/sas/data.txt

}


### Executable from this notebook

In [11]:
subprocess.run(['wsl', './sas.sh'], capture_output = True)

CompletedProcess(args=['wsl', './sas.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J--- v000001 ---\n--- v000002 ---\n--- v000003 ---\n--- v000004 ---\n--- v000005 ---\n--- v000006 ---\n--- v000007 ---\n--- v000008 ---\n--- v000009 ---\n--- v000010 ---\n--- v000011 ---\n--- v000012 ---\n--- v000013 ---\n--- v000014 ---\n--- v000015 ---\n--- v000016 ---\n--- v000017 ---\n--- v000018 ---\n--- v000019 ---\n--- v000020 ---\n--- v000021 ---\n--- v000022 ---\n--- v000023 ---\n--- v000024 ---\n--- v000025 ---\n--- v000026 ---\n--- v000027 ---\n--- v000028 ---\n--- v000029 ---\n--- v000030 ---\n--- v000031 ---\n--- v000032 ---\n--- v000033 ---\n--- v000034 ---\n--- v000035 ---\n--- v000036 ---\n--- v000037 ---\n--- v000038 ---\n--- v000039 ---\n--- v000040 ---\n--- v000041 ---\n--- v000042 ---\n--- v000043 ---\n--- v000044 ---\n--- v000045 ---\n--- v000046 ---\n--- v000047 ---\n--- v000048 ---\n--- v000049 ---\n--- v000050 ---\n--- v000051 ---\n--- v000052 ---\n--- v000053 ---\n--- v000054 -

## 'datamatrix' function

- Creates a piece of information called datamatrix

### Original function (Do not execute)

In [ ]:
datamatrix () {

mkdir -p images/temp

rm -f images/temp/*

cut -d' ' -f1 images/columns | uniq | sort > files

while read n word 
do
  echo "--- $n ---"
  rg -w $n images/columns | sort -t' ' -k1,1 > a
  echo "$n" > images/temp/$n
  join -a 1 -1 1 -2 1 -e 0 files a | sed "s/$/ $n 0/" | cut -d' ' -f3 >> images/temp/$n
done < images/selectedwords

echo "--- images/data.csv ...---"

awk '
        FNR==1 { col++ }
        FNR>max { max=FNR }
        { l[FNR,col]=$0 }
        END {
                for (i=1;i<=max;i++) {
                        for (j=1;j<=col;j++) {
                                printf "%-50s",l[i,j]
                        }
                        print ""
                }
        }
' images/temp/* > u
tr -s ' ' < u | tr ' ' ',' | sed 's/,$//' > images/data.csv

}


### Executable from this notebook

In [12]:
subprocess.run(['wsl', './datamatrix.sh'], capture_output = True)

CompletedProcess(args=['wsl', './datamatrix.sh'], returncode=0, stdout=b'\x1b[H\x1b[2J\x1b[3J--- v000001 ---\n--- v000002 ---\n--- v000003 ---\n--- v000004 ---\n--- v000005 ---\n--- v000006 ---\n--- v000007 ---\n--- v000008 ---\n--- v000009 ---\n--- v000010 ---\n--- v000011 ---\n--- v000012 ---\n--- v000013 ---\n--- v000014 ---\n--- v000015 ---\n--- v000016 ---\n--- v000017 ---\n--- v000018 ---\n--- v000019 ---\n--- v000020 ---\n--- v000021 ---\n--- v000022 ---\n--- v000023 ---\n--- v000024 ---\n--- v000025 ---\n--- v000026 ---\n--- v000027 ---\n--- v000028 ---\n--- v000029 ---\n--- v000030 ---\n--- v000031 ---\n--- v000032 ---\n--- v000033 ---\n--- v000034 ---\n--- v000035 ---\n--- v000036 ---\n--- v000037 ---\n--- v000038 ---\n--- v000039 ---\n--- v000040 ---\n--- v000041 ---\n--- v000042 ---\n--- v000043 ---\n--- v000044 ---\n--- v000045 ---\n--- v000046 ---\n--- v000047 ---\n--- v000048 ---\n--- v000049 ---\n--- v000050 ---\n--- v000051 ---\n--- v000052 ---\n--- v000053 ---\n--- v0

## 'correlationmatrix' function

- Calculates a correlation matrix from 'images/data.csv'

### Original function (Do not execute)

In [ ]:
correlationmatrix () {

echo "--- python correlation ... ---"

sed 's;data.csv;images/data.csv;' corr.py > p
python3 p > images/correlation

}


### Executable from this notebook
Note: If 'data.csv' is large, the execution of the following cell may be suspended to prevent the Python kernel from crashing. Instead, you can run the programme as 'corr.py' over Python over Ubuntu over Windows.

In [13]:
# Reference: https://www.geeksforgeeks.org/create-a-correlation-matrix-using-python/

# Create dataframe from file
dataframe = pd.read_csv('images/data.csv')

# Show dataframe
#print(dataframe)

# Use the corr() method on dataframe to create a correlation matrix
matrix = dataframe.corr()

# Print the correlation matrix
#print('The Correlation Matrix is: ')
#print(matrix)

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 8,
                       'display.width', 20000,
                       ):
    with open('images/correlation', 'w', encoding = 'utf8') as correlation:
        correlation.write(str(matrix))

#### Using Python over Ubuntu over Windows (Do not execute)

In [ ]:
eyamrog@RogLet-ASUS:~$ sudo apt update && sudo apt upgrade -y
[sudo] password for eyamrog:
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
All packages are up to date.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
eyamrog@RogLet-ASUS:~$ cd environments/my_env
eyamrog@RogLet-ASUS:~/environments/my_env$ source ./bin/activate
(my_env) eyamrog@RogLet-ASUS:~/environments/my_env$ pip freeze
aiohttp==3.8.5
aiosignal==1.3.1
asttokens==2.2.1
async-timeout==4.0.3
attrs==23.1.0
backcall==0.2.0
certifi==2023.7.22
charset-normalizer==3.2.0
click==8.1.6
decorator==5.1.1
executing==1.2.0
frozenlist==1.4.0
idna==3.4
ipython==8.14.0
jedi==0.19.0
joblib==1.3.2
matplotlib-inline==0.1.6
multidict==6.0.4
nltk==3.8.1
openai==0.27.8
parso==0.8.3
pexpect==4.8.0
pickleshare==0.7.5
prompt-toolkit==3.0.39
ptyprocess==0.7.0
pure-eval==0.2.2
Pygments==2.16.1
regex==2023.8.8
requests==2.31.0
six==1.16.0
stack-data==0.6.2
textblob==0.17.1
tqdm==4.66.1
traitlets==5.9.0
urllib3==2.0.4
wcwidth==0.2.6
yarl==1.9.2
(my_env) eyamrog@RogLet-ASUS:~/environments/my_env$ pip install pandas
Collecting pandas
  Downloading pandas-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 27.3 MB/s eta 0:00:00
Collecting python-dateutil>=2.8.2
  Downloading python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 KB 10.3 MB/s eta 0:00:00
Collecting pytz>=2020.1
  Downloading pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 KB 19.0 MB/s eta 0:00:00
Collecting tzdata>=2022.1
  Downloading tzdata-2023.3-py2.py3-none-any.whl (341 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 KB 18.0 MB/s eta 0:00:00
Collecting numpy>=1.22.4
  Downloading numpy-1.26.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 32.5 MB/s eta 0:00:00
Requirement already satisfied: six>=1.5 in ./lib/python3.10/site-packages (from python-dateutil>=2.8.2->pandas) (1.16.0)
Installing collected packages: pytz, tzdata, python-dateutil, numpy, pandas
Successfully installed numpy-1.26.0 pandas-2.1.1 python-dateutil-2.8.2 pytz-2023.3.post1 tzdata-2023.3
(my_env) eyamrog@RogLet-ASUS:~/environments/my_env$ pip freeze | grep pandas
pandas==2.1.1
(my_env) eyamrog@RogLet-ASUS:~/environments/my_env$ cp /mnt/c/Users/eyamr/Downloads/data.csv .
(my_env) eyamrog@RogLet-ASUS:~/environments/my_env$ cp /mnt/c/Users/eyamr/Downloads/corr.py .
(my_env) eyamrog@RogLet-ASUS:~/environments/my_env$ more corr.py
# Reference: https://www.geeksforgeeks.org/create-a-correlation-matrix-using-python/

import pandas as pd

# Create dataframe from file
dataframe = pd.read_csv('data.csv')

# Show dataframe
#print(dataframe)

# Use the corr() method on dataframe to create a correlation matrix
matrix = dataframe.corr()

# Print the correlation matrix
#print('The Correlation Matrix is: ')
#print(matrix)

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 8,
                       'display.width', 20000,
                       ):
    with open('correlation', 'w', encoding = 'utf8') as correlation:
        correlation.write(str(matrix))
(my_env) eyamrog@RogLet-ASUS:~/environments/my_env$ python corr.py
(my_env) eyamrog@RogLet-ASUS:~/environments/my_env$ ls -la
total 35676
drwxr-xr-x 8 eyamrog eyamrog    32768 Oct  7 10:00 .
drwxr-xr-x 3 eyamrog eyamrog     4096 Oct  3 11:25 ..
drwxr-xr-x 2 eyamrog eyamrog     4096 Oct  7 09:23 bin
-rwxr-xr-x 1 eyamrog eyamrog      718 Oct  7 09:58 corr.py
-rw-r--r-- 1 eyamrog eyamrog 12017004 Oct  7 10:00 correlation
-rwxr-xr-x 1 eyamrog eyamrog 24324000 Oct  7 09:26 data.csv
-rw-r--r-- 1 eyamrog eyamrog      250 Oct  4 16:33 files
drwxr-xr-x 2 eyamrog eyamrog    32768 Oct  4 16:26 images
drwxr-xr-x 2 eyamrog eyamrog     4096 Aug 17 13:28 include
drwxr-xr-x 2 eyamrog eyamrog    69632 Oct  4 16:33 labels
-rw-r--r-- 1 eyamrog eyamrog     6911 Oct  4 16:37 labels.zip
drwxr-xr-x 3 eyamrog eyamrog     4096 Aug 17 13:28 lib
lrwxrwxrwx 1 eyamrog eyamrog        3 Aug 17 13:28 lib64 -> lib
-rw-r--r-- 1 eyamrog eyamrog       71 Aug 17 13:28 pyvenv.cfg
drwxr-xr-x 3 eyamrog eyamrog     4096 Aug 20 10:50 share
-rwxr-xr-x 1 eyamrog eyamrog      225 Oct  3 12:43 vision.sh
(my_env) eyamrog@RogLet-ASUS:~/environments/my_env$ cp correlation /mnt/c/Users/eyamr/Downloads/
(my_env) eyamrog@RogLet-ASUS:~/environments/my_env$ deactivate
eyamrog@RogLet-ASUS:~/environments/my_env$ logout

## 'formats' function

- description

### Original function (Do not execute)

In [ ]:
formats () {

nlines=$( cat tweets/emoji.txt | wc -l | tr -dc '[0-9]' )

tail +2 images/correlation | tr -s ' ' | sed 's/^/CORR /' > bottom
head -1 images/correlation | tr -s ' ' | sed 's/^[ ]*//' | sed "s/\(v......\)/$nlines/g" | sed 's/^/N . /' > n

sed 's;data.csv;images/data.csv;' std.py > p
python3 p > s 
tr -s ' ' < s | cut -d' ' -f2 | grep -v 'float' | tr '\n' ' ' | sed 's/^/STD	 . /' > std 
echo >> std

sed 's;data.csv;images/data.csv;' mean.py > p
python3 p > m 
tr -s ' ' < m | cut -d' ' -f2 | grep -v 'float' | tr '\n' ' ' | sed 's/^/MEAN . /' > mean
echo >> mean

cat mean std n bottom > images/sas/corr.txt

echo "PROC FORMAT library=work ;
  VALUE  \$lexlabels" > images/sas/word_labels_format.sas
tr '\t' ' ' < images/selectedwords | sed 's/\(.*\) \(.*\)/"\1" = "\2"/' | sed -e 's/&/and/g' -e 's/%/pc/g' >> images/sas/word_labels_format.sas
echo ";
run;
quit;" >> images/sas/word_labels_format.sas

}


### Executable from this notebook

## 'examples' function

- description

### Original function (Do not execute)

In [ ]:
examples () {

mkdir -p images/examples
rm -f images/examples/*

html2text -nobs images/sas/output_group"$group"_images/loadtable.html > a

rm -f x??
split -p'=====' a
ls x?? > files

while read file
do
      pole=$( grep '^Factor ' $file | cut -d' ' -f2,3 | sed -e 's/^/f/' -e 's/ //g' )
      sed 's/^[ ]*//' $file | grep '^[0-9]' | tr -dc '[:alpha:][:punct:][0-9]\n ' | sed 's/^/~/' | tr  '[:space:]()' ' ' | tr -s ' ' |  tr '~' '\n' | cut -d' ' -f2 | grep -v '^$' | sed "s/^/$pole /" 
done < files > images/examples/factors


rm -f x??

head -1 images/sas/output_group"$group"_images/group4_images_scores.tsv | tr -d '\r' | tr '\t' '\n' > vars
    
last=$( cut -d' ' -f1 images/examples/factors | tr -dc '[0-9\n]' | sort -nr | head -1 )
    
for i in $(eval echo {1..$last});
do
      column=$( echo " $i + 1 " | bc ) 
      cut -f1,"$column" images/sas/output_group4_images/group4_images_scores_only.tsv  | tail +2 > a

      for pole in pos neg
      do
        echo "--- "f"$i""$pole"" ---" 

        if [ "$pole" == pos ] ; then
           sort -nr -k2,2 a | grep -v '\-' | tr '\t' ' ' | grep -v ' 0' | head -20 | nl -nrz > files
        else
           sort -n -k2,2 a | grep '\-' | tr '\t' ' ' | grep -v ' 0' | grep -v '	0' | head -20 | nl -nrz > files
        fi

        grep f"$i""$pole" images/examples/factors | sort -t' ' -k2,2 | cut -d' ' -f2 | sort > factor_words
        
        while read n file score
        do

          grep -m1 $file images/sas/output_group4_images/group4_images_scores.tsv | tr -d '\r' | tr '\t' '\n' > scores
          paste vars scores | tr '\t' ' ' | grep '^v' | grep -v ' 0$' | cut -d' ' -f1 | sort > vars_text
          join vars_text images/selectedwords | cut -d' ' -f2 | sort > vars_text_codes
          username=$( grep -w $file user_index.txt | cut -d' ' -f2 )
          picture=$( grep -w $file images/images_index.txt | cut -d'|' -f2,7 | tr ':|' ' ' | cut -d' ' -f2,4 | sed 's;\(.*\) \(.*\);\1.\2;' )
          folder=$( grep -w $file images/images_index.txt | cut -d'|' -f1 | tr ':|' ' ' | cut -d' ' -f2  )
          url=$( grep -m1 -B5 $file tweets/jq.txt | grep '"url"'  | cut -d':' -f2- | tr -d '",' | sed 's/^[ ]*//' )
          extension=$( echo $picture | cut -d'.' -f2 )
          cp images/images/$folder/$picture images/examples/image_f"$i""$pole"_x`"$n"."$extension"

          echo "---------------" 

          echo "# $n" 
          echo "score = $score"  
          echo "url: $url"
          echo

          grep -w $file images/labels.txt | tr '|' '\n' | sed 's/l:/~/' | tr '~' '\n'    

          echo
          echo "Lemmas in this picture that loaded on the factor:"
          echo

          join vars_text_codes factor_words > ll
          tr '\n' ',' < ll | sed 's/,/, /g' | sed 's/, $//' > images/examples/lemmas_f"$i"_"$pole"_"$n".txt
          cat ll

          echo 

        done < files > images/examples/examples_f"$i"_"$pole".txt

      done

done

    #rm -f vars factor_words scores vars_text vars_text_codes

}


### Executable from this notebook